[TOC](../toc.ipynb)

Advanced topics in ML
=====================

- KEYWORDS: jax




## Choice of activation functions in neural networks





The activation function in a neural network provides the nonlinearity in the model. We previously learned that one interpretation of the activation function is that it is a basis function that you can expand the data in to find a functional representation that fits the data.

Today we explore the impact of the activation function on the fitting, and extrapolation of neural networks. The following code is for setting up a neural network, and initializing the parameters with random numbers.





In [ ]:
import jax
from jax.config import config
config.update("jax_enable_x64", True)

import jax.numpy as np


def nn(params, X, activation=jax.nn.tanh):
    out = activation(X @ params['weights'][0] + params['biases'][0])

    for i in range(1, len(params['weights']) - 2):
        out = activation(out @ params['weights'][i] + params['biases'][i])

    # no activation on the output, just linear combination
    return out @ params['weights'][-1] + params['biases'][-1]


def init_params(scale, layers, seed=19):
    '''
    scale: float
    layers: integers for neurons in each layer
    '''
    key=jax.random.PRNGKey(seed)
    weights = []
    biases = []
    for i in range(len(layers) - 1):
        in_d = layers[i]
        out_d = layers[i + 1]

        weights += [jax.random.normal(key, (in_d, out_d)) * scale]
        biases += [jax.random.normal(key, (out_d,)) * scale]
        
    return dict(weights=weights, biases=biases)



In [ ]:
init_params(0.1, (2, 3, 4, 1))



As before, we are going to consider this dataset so we can evaluate fitting and extrapolation.





In [ ]:
# Some generated data
X = np.linspace(0, 1)[:, None]
Y = X ** (1.0 / 3.0)

import matplotlib.pyplot as plt

plt.plot(X, Y, "b.")
plt.xlabel("x")
plt.ylabel("y");



### tanh





First we review the case of `tanh` which is a classic activation function. The `tanh` function is "active" between about &plusmn; 2.5, and outside that window it saturates. That means the derivative of this function becomes close to zero outside that window. So if you have large values of inputs, you should scale them to avoid this issue.





In [ ]:
xt = np.linspace(-10, 10)
plt.plot(xt, np.tanh(xt))
plt.xlabel("x")
plt.ylabel("y");



In [ ]:
def objective(pars):
    p = nn(pars, X)
    errs = p - Y
    return (errs.T @ errs).squeeze()

import jax.example_libraries.optimizers as jax_opt

from jax import jit, value_and_grad

opt_init, opt_update, get_params = jax_opt.adam(1e-2)

params1 = init_params(0.1, (1, 5, 1), seed=42)
opt_state = opt_init(params1)

@jit
def step(i, opt_state):
    value, grads = value_and_grad(objective)(get_params(opt_state))
    return value, opt_update(i, grads, opt_state)

for i in range(10000):
    value, opt_state = step(i, opt_state)
    if np.abs(value) < 1e-4:
        break



Now we can examine the fit and extrapolation.





In [ ]:
X2 = np.linspace(-2, 5)
Y2 = X2 ** (1 / 3)
Z2 = nn(get_params(opt_state), X2[:, None])

plt.plot(X2, Y2, "b.", label="analytical")
plt.plot(X2, Z2, label="NN")
plt.fill_between(X2 < 1, 0, 1.4, facecolor="gray", alpha=0.5)
plt.xlabel("x")
plt.ylabel("y");



For large enough $x$, all of the `tanh` functions saturate at $y=1$. So, the neural network also saturates at a constant value for large $x$.

**exercise** Can you work out from the NN math what the saturated values should be?





### relu





A common activation function in deep learning is the Relu:





In [ ]:
def relu(x):
    return x * (x > 0)


plt.plot(X2, relu(X2));



This is popular because if is very fast to compute, and the derivatives are constant. For positive $x$ there is no saturation. For negative $x$, however, the neuron is "dead".





In [ ]:
def objective2(par, step=None):
    pred = nn(par, X, activation=relu)
    err = Y - pred
    return (err.T @ err).squeeze()

params2 = init_params(0.01, [1, 3, 1], seed=2)
opt_state = opt_init(params2)

@jit
def step(i, opt_state):
    value, grads = value_and_grad(objective2)(get_params(opt_state))
    return value, opt_update(i, grads, opt_state)

for i in range(10000):
    value, opt_state = step(i, opt_state)
    if np.abs(value) < 1e-4:
        break
        
params2 = get_params(opt_state)



In [ ]:
X2 = np.linspace(0.0, 1)
Y2 = X2 ** (1 / 3)
Z2 = nn(params2, X2[:, None], activation=relu)

plt.plot(X2, Y2, "b.", label="analytical")
plt.plot(X2, Z2, label="NN")
plt.xlabel("x")
plt.ylabel("y");



Notes:

1.  The fit is not very good.
2.  we have piecewise linear fits here.
3.  There are negative weights, which means there are some "dead neurons". Maybe other initial guesses might improve this.

Let's look at the extrapolating behavior.





In [ ]:
X2 = np.linspace(0, 1)
Y2 = X2 ** (1 / 3)

xf = np.linspace(-2, 2)
Z2 = nn(params2, xf.reshape([-1, 1]), activation=relu)

plt.plot(X2, Y2, "b.", label="analytical")
plt.plot(xf, Z2, label="NN")
plt.fill_between(X2 < 1, 0, 1.4, facecolor="gray", alpha=0.5)
plt.xlabel("x")
plt.ylabel("y");



Note this extrapolates linearly on the right, and is constant on the left. These are properties of the Relu.





### Gaussian (radial basis function)





Finally we consider the Gaussian activation function.





In [ ]:
def rbf(x):
    return np.exp(-(x**2))


x3 = np.linspace(-3, 3)
plt.plot(x3, rbf(x3));



Now we fit the data.





In [ ]:
def objective3(pars, step=None):
    pred = nn(pars, X, activation=rbf)
    err = Y - pred
    return np.mean(err**2)

params3 = init_params(0.01, [1, 3, 1], seed=2)
opt_state = opt_init(params3)

@jit
def step(i, opt_state):
    value, grads = value_and_grad(objective3)(get_params(opt_state))
    return value, opt_update(i, grads, opt_state)

for i in range(10000):
    value, opt_state = step(i, opt_state)
    if np.abs(value) < 1e-4:
        break
        
params3 = get_params(opt_state)



In [ ]:
X2 = np.linspace(0.0, 1)
Y2 = X2 ** (1 / 3)
Z2 = nn(params3, X2.reshape([-1, 1]), activation=rbf)

plt.plot(X2, Y2, "b.", label="analytical")
plt.plot(X2, Z2, label="NN")
plt.xlabel("x")
plt.ylabel("y");



And we look at the extrapolation behavior again.



In [ ]:
X2 = np.linspace(-2.5, 5)
Y2 = X2 ** (1 / 3)
Z2 = nn(params3, X2.reshape([-1, 1]), activation=rbf)

plt.plot(X2, Y2, "b.", label="analytical")
plt.plot(X2, Z2, label="NN")
plt.fill_between(X2 < 1, 0, 1.4, facecolor="gray", alpha=0.5)
plt.xlabel("x")
plt.ylabel("y");



Note this extrapolates to zero when you are far from the data. It fits reasonably in the region trained. "If your function is nonlinear enough, somewhere the nonlinearity matches your data." (Z. Ulissi).





### Summary





We can think of single layer neural networks as partial expansions in the activation function space. That means the extrapolation behavior will be like the dominating feature of the activation functions, e.g. relu extrapolates like a line, tanh saturates at large x, and Gaussians effectively go to zero. Unexpected things can happen at the edges of the data, so at intermediate extrapolations you do not always know what will happen.





## Train/test splits on data





So far we have not considered how to split your data when fitting. This becomes important for a few reasons:

1.  We need to be able to tell if we are overfitting. One way to do this is to compare fitting errors to prediction errors.

This means we need a way to split a dataset into a train set and a test set. Then, we can do training on the train set, and testing on the test set.

Let's start by remembering what our dataset is.





In [ ]:
X = np.linspace(0, 1)
Y2 = X ** (1 / 3)
X



The way to split this is that we use indexing. We start by making an array of integers.





In [ ]:
ind = np.arange(len(X))
ind



In [ ]:
X[ind]  # we can use an array of integers for indexing



Next, we randomly shuffle the array of integers.





In [ ]:
key = jax.random.PRNGKey(22)
pind = jax.random.permutation(key, ind)
pind



Next, we decide on the train/test split. A common choice is 80/20. We find the integer that is closest to 80% of the index array.





In [ ]:
split = int(0.8 * len(pind))
split



In [ ]:
train_ind = pind[:split]
test_ind = pind[split:]
(len(train_ind), len(test_ind))



We check that we have a reasonable choice here.





In [ ]:
train_x = X[train_ind]
train_y = Y2[train_ind]

test_x = X[test_ind]
test_y = Y2[test_ind]
plt.plot(test_x, test_y, "ro", label="test")
plt.plot(train_x, train_y, "bo", label="train")
plt.legend();



Now, we train on the train data.





In [ ]:
def objective10(params, step=None):
    pred = nn(params, np.array([train_x]).T)
    err = np.array([train_y]).T - pred
    return np.mean(err**2)

params10 = init_params(0.01, [1, 3, 1], seed=2)
opt_state = opt_init(params10)

@jit
def step(i, opt_state):
    value, grads = value_and_grad(objective10)(get_params(opt_state))
    return value, opt_update(i, grads, opt_state)

for i in range(10000):
    value, opt_state = step(i, opt_state)
    if np.abs(value) < 1e-4:
        break
        
params10 = get_params(opt_state)



As usual, we should check the fit on the train data. This is a little trickier than before, because the points are out of order.





In [ ]:
Z2 = nn(params10, train_x[:, None])
plt.plot(train_x, Z2, "bo", label="NN")
plt.plot(train_x, train_y, "r.", label="analytical")
plt.xlabel("x")
plt.ylabel("y")

plt.plot(test_x, nn(params10, test_x[:, None]), "go", label="NN")
plt.plot(test_x, test_y, "y.", label="analytical");



In [ ]:
# root mean squared error
rmse_train = np.mean((train_y - nn(params10, train_x[:, None])) ** 2)
rmse_test = np.mean((test_y - nn(params10, test_x[:, None])) ** 2)

print(f"""RMSE train = {rmse_train:1.3f}
RMSE test = {rmse_test:1.3f}""")



For the train/test split to be meaningful, it is important that the two datasets have similar distributions of values.





## Summary





Today we reviewed the role of activation functions in neural networks, and observed that it doesn't generally matter what you use (but the details always matter in individual cases). The mathematical form of these activation functions determines how they will extrapolate, which can be important depending on your application.

We then explored how to efficiently split a dataset into a train and test set so that overfitting can be evaluated. This becomes increasingly important for when you plan to explore many models (choices of hyperparameters), and then you split the data three ways (train, test and validate).



